# Лабораторная работа №2.Поиск ассоциативных правил(Last FM hometask)
https://www.kaggle.com/ravichaubey1506/lastfm <br>
## Зямзина Анастасия и Лукашева Екатерина
 <b>3530903_70302</b>
 
Задание: 
1. Обработать данные по странам своей группы (совместно): Spain, Portugal, France, Italy, Belgium<br> 
2. Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.
3. Вывести эти правила в отдельных ячейках. 
4. Подумать, как можно было бы использовать полученные правила.

## Подготовка данных

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("lastfm.csv")
data.head()

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany


In [3]:
countries = ['Spain', 'Portugal', 'France', 'Italy', 'Belgium']
data_countries = pd.DataFrame()
for i in range(len(countries)):
    df2 = data[data.country == countries[i]]
    data_countries= pd.concat([data_countries, df2], ignore_index=True)
data_countries.country.value_counts()

Spain       9322
France      5962
Italy       5717
Belgium     3331
Portugal    2882
Name: country, dtype: int64

## Узнаем, какие артисты нравятся каждому пользователю

In [4]:
data_countries.describe()
data_by_user = data_countries.groupby(['user','country'])['artist'].apply(lambda x: ",".join(x)).reset_index()
data_by_user

,user,country,artist
0,6,Portugal,"lily allen,kanye west,sigur rós,pink floyd,ste..."
1,12,Italy,"queen,the beatles,lynyrd skynyrd,neil young,th..."
2,36,Italy,"m.i.a.,fugazi,johnny cash,clint mansell,elton ..."
3,47,Italy,"pj harvey,radiohead,zero 7,einstürzende neubau..."
4,74,Spain,"in extremo,nine inch nails,billy talent,panic!..."
...,...,...,...
1414,19690,Italy,"pink floyd,the beatles,the white stripes,depec..."
1415,19693,Belgium,"air,simon & garfunkel,pixies,serge gainsbourg,..."
1416,19697,Belgium,"underworld,pendulum,the prodigy,sufjan stevens..."
1417,19704,Italy,"keane,pinback,coldplay,travis,calexico,doves,d..."


In [5]:
data_by_user['country'].value_counts()

Spain       506
France      327
Italy       304
Belgium     160
Portugal    122
Name: country, dtype: int64

## Подготовка данных для обработки алгоримами (dummy encoding)

In [46]:
for_dummy_enc = data_countries.drop(columns=['country','sex'])
data_to_process = for_dummy_enc.pivot_table(index='user', columns='artist',aggfunc=any, fill_value=False).astype(int)

for_dummy_enc_with_sex = data_countries.drop(columns=['artist','country'])
data_to_process_with_sex = pd.concat([data_to_process, 
                                      for_dummy_enc_with_sex.pivot_table(index='user', columns='sex',aggfunc=any, fill_value=False).astype(int)], 
                                     axis = 1)

## Обработка данных различными алгоритмами

#### Apriori

In [7]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets_apriori = apriori(data_to_process, min_support=0.01, use_colnames=True)

frequent_itemsets_apriori.head()

,support,itemsets
0,0.011276,(2pac)
1,0.020437,(3 doors down)
2,0.029598,(30 seconds to mars)
3,0.012685,(50 cent)
4,0.019732,(65daysofstatic)


#### FP-growth

In [8]:
from mlxtend.frequent_patterns import fpgrowth

frequent_itemsets_fpgrowth = fpgrowth(data_to_process, min_support=0.08, use_colnames=True)

frequent_itemsets_fpgrowth.head()

,support,itemsets
0,0.174066,(muse)
1,0.130374,(pink floyd)
2,0.123326,(metallica)
3,0.096547,(depeche mode)
4,0.088090,(u2)


#### FPMax

In [9]:
from mlxtend.frequent_patterns import fpmax

frequent_itemsets_fpmax = fpmax(data_to_process, min_support=0.05, use_colnames=True)

frequent_itemsets_fpmax

,support,itemsets
0,0.050035,(gorillaz)
1,0.050035,(pj harvey)
2,0.050035,(editors)
3,0.050740,(kaiser chiefs)
4,0.050740,(snow patrol)
...,...,...
67,0.054968,"(radiohead, pink floyd)"
68,0.070472,"(the beatles, radiohead)"
69,0.068358,"(coldplay, muse)"
70,0.080338,"(radiohead, muse)"


## Правила 

Дополнтельные функции

In [28]:
def print_res(algorithm, answer):
    print('Algorithm: ' + algorithm)
    for i in range(len(answer['antecedents'])):
        print(set(answer.iloc[i,0]) , ' -> ', set(answer.iloc[i,1]))

In [33]:
def first_rule(data_for_this_rule, min_t):
    return association_rules(data_for_this_rule, metric="support", min_threshold=min_t) 

data_first_rule_apriori = first_rule(frequent_itemsets_apriori,0.005)
print(data_first_rule_apriori.head())
data_first_rule_fpgrowth = first_rule(frequent_itemsets_fpgrowth, 0.001)
print(data_first_rule_fpgrowth.head())

            antecedents           consequents  antecedent support  \
0        (fall out boy)  (30 seconds to mars)            0.038760   
1  (30 seconds to mars)        (fall out boy)            0.029598   
2  (30 seconds to mars)         (linkin park)            0.029598   
3         (linkin park)  (30 seconds to mars)            0.083862   
4  (30 seconds to mars)                (muse)            0.029598   

   consequent support   support  confidence      lift  leverage  conviction  
0            0.029598  0.010571    0.272727  9.214286  0.009424    1.334302  
1            0.038760  0.010571    0.357143  9.214286  0.009424    1.495263  
2            0.083862  0.014799    0.500000  5.962185  0.012317    1.832276  
3            0.029598  0.014799    0.176471  5.962185  0.012317    1.178345  
4            0.174066  0.012685    0.428571  2.462117  0.007533    1.445384  
   antecedents  consequents  antecedent support  consequent support   support  \
0  (radiohead)       (muse)         

In [42]:
def first_rule(data, sup, lift):
    response = data[
        (data['antecedent support'] >= sup) &
        (data['lift'] >= lift) &
        (data['antecedents'] == {'radiohead'})
        
    ]
    return response

"""
    Для популярной группы/исполнителя подберем подобрать похожие группы/артистов
"""
rules1 = first_rule(data_first_rule_apriori,0.05,3)
rules1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3149,(radiohead),(thom yorke),0.237491,0.015504,0.011980,0.050445,3.253709,0.008298,1.036797
3435,(radiohead),"(air, beck)",0.237491,0.013390,0.010571,0.044510,3.324223,0.007391,1.032570
3518,(radiohead),"(air, nirvana)",0.237491,0.018323,0.013390,0.056380,3.077037,0.009038,1.040331
3578,(radiohead),"(coldplay, antony and the johnsons)",0.237491,0.015504,0.011276,0.047478,3.062315,0.007594,1.033568
3613,(radiohead),"(arcade fire, david bowie)",0.237491,0.012685,0.011276,0.047478,3.742829,0.008263,1.036527
3949,(radiohead),"(death cab for cutie, arctic monkeys)",0.237491,0.018323,0.013390,0.056380,3.077037,0.009038,1.040331
4377,(radiohead),"(beck, portishead)",0.237491,0.012685,0.010571,0.044510,3.508902,0.007558,1.033308
4417,(radiohead),"(muse, björk)",0.237491,0.016913,0.013390,0.056380,3.333457,0.009373,1.041825
4423,(radiohead),"(placebo, björk)",0.237491,0.014799,0.010571,0.044510,3.007630,0.007056,1.031095
4435,(radiohead),"(sigur rós, björk)",0.237491,0.019027,0.014094,0.059347,3.119024,0.009576,1.042864


In [43]:
get_max_confid = rules1['confidence'].max()
rules1[
     (rules1['confidence'] >= get_max_confid) 
]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6644,(radiohead),"(interpol, franz ferdinand)",0.237491,0.029598,0.022551,0.094955,3.208139,0.015522,1.072214


In [44]:
first_rule(data_first_rule_fpgrowth,0.05,1.5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(radiohead),(muse),0.237491,0.174066,0.080338,0.338279,1.943392,0.038999,1.248160
3,(radiohead),(coldplay),0.237491,0.188161,0.083157,0.350148,1.860901,0.038471,1.249269


#### 5 правило:

In [50]:
def data_for_fifth_rule(data_for_this_rule):
    new_data = association_rules(data_for_this_rule, metric="support", min_threshold=0.001) 
    new_data["antecedent_len"] = new_data["antecedents"].apply(lambda x: len(x))
    new_data["consequent_len"] = new_data["consequents"].apply(lambda x: len(x))
    return new_data 

frequent_itemsets_fpgrowth = fpgrowth(data_to_process_with_sex, min_support=0.005, use_colnames=True)


data_fifth_rule_apriori = data_for_fifth_rule(frequent_itemsets_apriori)
print(data_fifth_rule_apriori.head())

data_fifth_rule_fpgrowth = data_for_fifth_rule(frequent_itemsets_fpgrowth)
print(data_fifth_rule_fpgrowth.head())

            antecedents           consequents  antecedent support  \
0        (fall out boy)  (30 seconds to mars)            0.038760   
1  (30 seconds to mars)        (fall out boy)            0.029598   
2  (30 seconds to mars)         (linkin park)            0.029598   
3         (linkin park)  (30 seconds to mars)            0.083862   
4  (30 seconds to mars)                (muse)            0.029598   

   consequent support   support  confidence      lift  leverage  conviction  \
0            0.029598  0.010571    0.272727  9.214286  0.009424    1.334302   
1            0.038760  0.010571    0.357143  9.214286  0.009424    1.495263   
2            0.083862  0.014799    0.500000  5.962185  0.012317    1.832276   
3            0.029598  0.014799    0.176471  5.962185  0.012317    1.178345   
4            0.174066  0.012685    0.428571  2.462117  0.007533    1.445384   

   antecedent_len  consequent_len  
0               1               1  
1               1               1  
2 

In [62]:
def fifth_rule(data,sex):
    response = data[ (data['antecedents']== sex) 
                    & (data['antecedent_len'] == 1) 
                    & (data['consequent_len'] == 1) 
                    & (data['consequent support'] > 0.05)
                    & (data['leverage'] > 0.01)
                   ]
    return response

"""
 Определить популярную группу у определенного пола
"""    
male = frozenset('m')
female = frozenset('f')

fifth_rule(data_fifth_rule_fpgrowth, male)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
178,(m),(pink floyd),0.780127,0.130374,0.112051,0.143631,1.101692,0.010343,1.015482,1,1
535,(m),(metallica),0.780127,0.123326,0.109232,0.140018,1.135346,0.013022,1.019409,1,1


In [63]:
fifth_rule(data_fifth_rule_fpgrowth,female)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
91824,(f),(amy winehouse),0.219873,0.090204,0.030303,0.137821,1.527870,0.01047,1.055228,1,1
173660,(f),(placebo),0.219873,0.098661,0.031712,0.144231,1.461882,0.01002,1.053250,1,1
